# 导入

In [1]:
from utils import MongoConnection, get_config
import numpy as np
import jieba
import re

# 链接数据库

In [2]:
config_path = './config.ini'
data_db_name = 'sspai'
db_name = get_config(config_path=config_path, section=data_db_name, key='db_name')
collection_name = get_config(config_path=config_path, section=data_db_name, key='collection_name')
mongo_connection = MongoConnection(db_name, collection_name)

# 获取所有content
共有20972条数据

In [3]:
all_content = mongo_connection.find({}, {'content': 1, '_id': 0})
all_content = [x['content'] for x in all_content]

In [4]:
len(all_content)

20972

# 统计每篇的句子数
根据四分位数，获取上限（非异常范围内的最大值）为75（74.5）

In [5]:
sentence_len = [len(x) for x in all_content]
q1, q2, q3 = np.percentile(sentence_len, [25, 50, 75], interpolation='midpoint')
q1, q2, q3

(7.0, 18.0, 34.0)

In [6]:
upper = q3 + 1.5 * (q3 - q1)
upper

74.5

# 统计每个句子的字符数
根据四分位数，获取上限（非异常范围内的最大值）为116（115.5）

In [7]:
def content_cut(content):
    cop = re.compile("[^\u4e00-\u9fa5^a-z^A-Z^0-9]")
    stop_words = [' ', '丨']  # 停用词
    content = [cop.sub('', x) for x in content]
    cut_contents = [jieba.cut(x, cut_all=False) for x in content]
    cut_contents = [[y for y in x] for x in cut_contents]
#     words = []
#     for cut_content in cut_contents:
#         words.extend(cut_content)
    words = [[y for y in x if y not in stop_words] for x in cut_contents]
    return words

In [8]:
cut_words = [content_cut(x) for x in all_content]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\AMBROS~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.544 seconds.
Prefix dict has been built succesfully.


In [12]:
words_counts = [[len(y) for y in x] for x in cut_words]
word_count = []
for single_word_count in words_counts:
    word_count.extend(single_word_count)
word_count[:3]

[2, 50, 16]

In [13]:
q1, q2, q3 = np.percentile(word_count, [25, 50, 75], interpolation='midpoint')
q1, q2, q3

(8.0, 27.0, 51.0)

In [14]:
upper = q3 + 1.5 * (q3 - q1)
upper

115.5